In [1]:
import pandas as pd
from itertools import combinations
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.api import VAR
# from statsmodels.tsa.vector_ar.var_model import VAR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from datetime import timedelta

In [2]:
df = pd.read_csv("../data/csv/Gold_Data.csv",index_col="Business_Week")
# print(len(df.columns),len(df.index))

In [3]:
# df = df.drop(df.columns[0],axis=1)
# df = df.drop(df.columns[7],axis=1)
# df = df.dropna(axis=1)
df = df.dropna()
# print(len(df.columns),len(df.index))

In [4]:
# df.iloc[25:30]

In [5]:
# Freq is very important, specifies when and how often data is taken.
# VAR will not work without., in terms of specifying linear time for input.
df["Date"] = pd.to_datetime(df["Date"],infer_datetime_format=True)
df = df.reset_index(drop=True)
for i in range(len(df["Date"])):
    if df["Date"][i].weekday() != 1:
#         print(df["Date"][i])
        try:
            df.at[i,"Date"] = df["Date"][i-1]+timedelta(days=7)
#             print(df["Date"][i])
#             print(df["Date"][i].weekday())
        except:
            df.at[i,"Date"] = df["Date"][i+1]-timedelta(days=7)
# print(df.iloc[29,:])
df.set_index("Date",inplace=True)
df = df.asfreq("W-TUE")

In [6]:
df = df.dropna()
# len(df.index)

In [7]:
len(list(df.columns))

23

In [8]:
check_columns = list(df.columns)
check_columns.remove("Gold_Settle")

In [9]:
test_columns = [list(combination) for combination in combinations(check_columns,12)]
test_results = [coint_johansen(df[combination],-1,1) for combination in test_columns]

In [11]:
# len(test_results[12].evec)

In [12]:
# test_results[12].lr1

In [13]:
# test_results[12].cvt

In [10]:
cointegration_ranking = [sum([1 if result.lr1[i] >= result.cvt[i][2] else 0 for i in range(12)]) for result in test_results]

In [11]:
# Will break if all 12 features tested have cointegration, this is real unlikely though.
highest_coint_rank = max(cointegration_ranking)
# print(highest_coint_rank)
coint_info_index = cointegration_ranking.index(highest_coint_rank)
coint_vectors = [test_columns[coint_info_index].copy(),list(test_results[coint_info_index].evec[highest_coint_rank])]
for i in range(highest_coint_rank+1,12):
    coint_vectors[0].pop(i)
    coint_vectors[1].pop(i)

In [23]:
test_results[coint_info_index].eig

array([0.58614936, 0.49468256, 0.47595059, 0.41830721, 0.33463355,
       0.2919888 , 0.2032198 , 0.18628807, 0.15566185, 0.12072655,
       0.11254034, 0.00431322])

In [12]:
modulated_series = [df[coint_vectors[0][i]]*coint_vectors[1][i] for i in range(highest_coint_rank+1)]

In [13]:
stationary_series = modulated_series.pop(0)
for series in modulated_series:
    stationary_series += series

In [14]:
df["Stationary_Conglomerate"] = stationary_series

In [15]:
df.drop(coint_vectors[0],axis=1,inplace=True)

In [16]:
df.tail()

,Gold_Settle,Producer/Merchant/Processor/User Longs,Producer/Merchant/Processor/User Shorts,Ten_Yr_Futures_Settle,Ten_Yr_Futures_Prev. Day Open Interest,USD_Settle,USD_Volume,USD_Prev. Day Open Interest,ZAR_Volume,ZAR_Wave,ZAR_Prev. Day Open Interest,Stationary_Conglomerate
Date,,,,,,,,,,,,
2019-03-12,1290.16,43255.0,105756.0,122.378125,3825284.6,97.1538,22297.0,57772.0,25.8,14.368210,66.750000,-229711.443535
2019-03-19,1301.60,51053.0,118282.0,122.568750,3885013.6,96.3370,14723.6,52886.2,20.4,14.495198,106.000000,-235144.483403
2019-06-04,1303.92,41139.0,124446.0,126.234375,4184701.6,97.5626,18806.4,49452.4,11.8,14.652196,109.250000,188283.148924
2019-06-11,1331.64,46038.0,136187.0,126.603125,3965213.6,96.8188,25410.2,44996.2,12.0,14.895546,103.200000,-222303.419248
2019-06-18,1339.30,52208.0,151171.0,126.937500,3940941.0,97.1488,20748.8,47139.4,9.5,14.848470,132.666667,-259882.785614


In [17]:
scaler = MinMaxScaler()

In [18]:
normalized_df = scaler.fit_transform(df)
normalized_df = pd.DataFrame(normalized_df,index=df.index,columns=df.columns)
normalized_df.tail()

,Gold_Settle,Producer/Merchant/Processor/User Longs,Producer/Merchant/Processor/User Shorts,Ten_Yr_Futures_Settle,Ten_Yr_Futures_Prev. Day Open Interest,USD_Settle,USD_Volume,USD_Prev. Day Open Interest,ZAR_Volume,ZAR_Wave,ZAR_Prev. Day Open Interest,Stationary_Conglomerate
Date,,,,,,,,,,,,
2019-03-12,0.817106,0.247051,0.178973,0.247789,0.728439,0.755093,0.169370,0.228298,0.035144,0.896285,0.001759,0.019953
2019-03-19,0.859684,0.381321,0.258225,0.262776,0.759372,0.719870,0.071121,0.183342,0.027492,0.905234,0.005562,0.016360
2019-06-04,0.868319,0.210617,0.297225,0.550983,0.914578,0.772722,0.124087,0.151746,0.015305,0.916298,0.005877,0.296386
2019-06-11,0.971490,0.294970,0.371511,0.579975,0.800907,0.740647,0.209757,0.110743,0.015588,0.933448,0.005290,0.024852
2019-06-18,1.000000,0.401209,0.466315,0.606265,0.788336,0.754877,0.149285,0.130463,0.012045,0.930131,0.008146,0.000000


In [19]:
train = normalized_df.iloc[:int(.8*(len(normalized_df.index)))]
test = normalized_df.iloc[int(.8*(len(normalized_df.index))):]

In [20]:
model = VAR(endog=train)

C:\Users\benel\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


# https://github.com/scikit-learn/scikit-learn/issues/2640

There are roundoff errors with the model when flitering the data in this manner. This doesn't seem to offer an easy fix and is likely to fail like the other model.
Not Worth Pursuing Further

In [21]:
fit_model = model.fit(maxlags=15, ic='aic')

LinAlgError: 11-th leading minor of the array is not positive definite